In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow-addons

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Dropout, MaxPool2D,#Normalization,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add
)
import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split

In [ ]:
img_shape = (28,28,1)
epochs = 100
batch = 16

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
testing = test.to_numpy().reshape(28000,28,28,1)

In [ ]:
x = train.iloc[:,1:]
y = train.iloc[:,:1]

In [ ]:
train_x, val_x, train_y,val_y = train_test_split(x,y, test_size=0.2,
                                                stratify=y)

print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)

In [ ]:
def gen(data_x,data_y):
    for x in range(len(data_x)):
        y = np.zeros(10)
        tmp = data_x[x]
        input_ = tmp.reshape(img_shape)/255.
        y[int(data_y[x][0])] = 1.
        input_ = tf.convert_to_tensor(input_,dtype=tf.float32)
        y = tf.convert_to_tensor(y,dtype=tf.float32)
        yield input_,y

In [ ]:
train_dataset = tf.data.Dataset.from_generator(gen,
                                         (tf.float32,tf.float32),
                                         ((img_shape),(10)),
                                         args=(train_x,train_y))
val_dataset = tf.data.Dataset.from_generator(gen,
                                         (tf.float32,tf.float32),
                                         (img_shape,(10)),
                                         args=(val_x,val_y))
train_dataset = train_dataset.shuffle(128).batch(batch)
val_dataset = val_dataset.shuffle(128).batch(batch)
for example in train_dataset.take(1):
    plt.subplot(2,1,1)
    image, label = example
    plt.imshow(image[0])
    plt.title(str(np.argmax(label)))
    
for example in val_dataset.take(1):
    plt.subplot(2,1,2)
    image, label = example
    plt.imshow(image[0])
    plt.title(str(np.argmax(label)))    
    
plt.show()

## Basic CNN

In [ ]:
def makemodel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
cnn_model = makemodel()
cnn_history = cnn_model.fit(
    train_dataset,validation_data=val_dataset,
    epochs=epochs, 
    verbose=1
)

* * *

## VIT  
[reference](https://keras.io/examples/vision/image_classification_with_vision_transformer/)

In [ ]:
input_shape = (28,28,1)
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 16
num_epochs = 100
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
def gen(data_x,data_y):
    for x in range(len(data_x)):
        y = np.zeros(10)
        tmp = data_x[x]
        input_ = tmp.reshape(img_shape)/255.
        y[int(data_y[x][0])] = 1.
        input_ = tf.convert_to_tensor(input_,dtype=tf.float32)
        y = tf.convert_to_tensor(y,dtype=tf.float32)
        yield input_,data_y[x]
        
train_dataset = tf.data.Dataset.from_generator(gen,
                                         (tf.float32,tf.float32),
                                         ((img_shape),(1)),
                                         args=(train_x,train_y))
val_dataset = tf.data.Dataset.from_generator(gen,
                                         (tf.float32,tf.float32),
                                         (img_shape,(1)),
                                         args=(val_x,val_y))
train_dataset = train_dataset.shuffle(128).batch(batch_size)
val_dataset = val_dataset.shuffle(128).batch(batch_size)

for image, label in train_dataset.take(1):
    print(label)
    for i in range(batch_size):
        plt.subplot(4,4,i+1)
        plt.imshow(image[i])

plt.show()

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.Resizing(image_size, image_size),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(factor=0.02),
        layers.experimental.preprocessing.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
plt.figure(figsize=(4, 4))
image = train_x.iloc[0].to_numpy().reshape((28,28,1))
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 1))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(10)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "./"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train_dataset,validation_data=val_dataset,
        batch_size=batch_size,
        epochs=num_epochs,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(val_dataset)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    return history

In [ ]:
vit_classifier = create_vit_classifier()
vit_history = run_experiment(vit_classifier)

In [ ]:
model = create_vit_classifier()
model.load_weights('./')

In [ ]:
pred = model.predict(testing)

In [ ]:
preds = []
for x in pred:
    preds.append(np.argmax(x))

In [ ]:
test = test.to_numpy()
def gen(data_x):
    for x in range(len(data_x)):
        y = np.zeros(10)
        tmp = data_x[x]
        input_ = tmp.reshape(img_shape)/255.
        y[int(data_y[x][0])] = 1.
        input_ = tf.convert_to_tensor(input_,dtype=tf.float32)
        y = tf.convert_to_tensor(y,dtype=tf.float32)
        yield input_
        
test_dataset = tf.data.Dataset.from_generator(gen,
                                         (tf.float32),
                                         (img_shape),
                                         args=(test))
train_dataset = train_dataset.batch(batch)

In [ ]:
sub.to_csv('sub.csv')